In [ ]:
import numpy
from dask import dataframe
from matplotlib import pyplot as plt

In [ ]:
def wave(time, *, sigma=0.25):
    return numpy.cos(2.0*numpy.pi*10*time) + sigma*numpy.random.randn(*time.shape)

In [ ]:
size = 256
time = numpy.linspace(0.0, 1.0, 1024)

samples = wave(numpy.asarray(256*[time]))

plt.figure(figsize=(12, 3))

for sample in samples:
    plt.scatter(time, sample, marker='x', color='gray', alpha=0.1)
    
plt.plot(time, samples.mean(0), color='black')

plt.tight_layout()
plt.show()

In [ ]:
for i, sample in enumerate(samples):
    numpy.savetxt(f'sample_{i:03}.csv', numpy.stack([time, sample]).T, delimiter=',', header='time,voltage', comments='')

In [ ]:
!ls sample_*.csv | wc -l

In [ ]:
!head -n 10 sample_000.csv

In [ ]:
df = dataframe.read_csv('sample_*.csv')
df

In [ ]:
df.head()

In [ ]:
task = df.groupby('time').mean()
task

In [ ]:
repr(task)

In [ ]:
result = task.compute()
result

In [ ]:
plt.figure(figsize=(12, 3))

for sample in samples:
    plt.scatter(time, sample, marker='x', color='gray', alpha=0.1)
    
plt.plot(time, result.voltage, color='black')

plt.tight_layout()
plt.show()

In [ ]:
window = numpy.exp(-1.0/((1.0 - time)*time))

plt.figure(figsize=(12, 3))
plt.plot(time, window, color='black')

plt.tight_layout()
plt.show()

In [ ]:
from dask import delayed
from dask import compute

from time import sleep

def process(file, window):
    sleep(0.1)
    _, voltage = numpy.loadtxt(file, delimiter=',', skiprows=1).T
    return voltage*window

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(time, process('sample_000.csv', window), color='black')
plt.tight_layout()
plt.show()

In [ ]:
from pathlib import Path

files = [path.name for path in Path().glob('sample_*.csv')]

In [ ]:
%%time

todos = [delayed(lambda file: process(file, window))(file) for file in files]

In [ ]:
%%time

# "threads", "synchronous" or "processes"

samples, *_ = compute(todos, scheduler='synchronous')

In [ ]:
plt.figure(figsize=(12, 3))

for sample in samples:
    plt.scatter(time, sample, marker='x', color='gray', alpha=0.1)

plt.tight_layout()
plt.show()

In [ ]:
# https://github.com/dask/dask-tutorial